## Предсказания модели

In [1]:
import numpy as np
import pandas as pd
import dill

# from sklearn.base import BaseEstimator, TransformerMixin
# from sklearn.pipeline import Pipeline, FeatureUnion
# from sklearn.model_selection import train_test_split
# from xgboost import XGBClassifier
from sklearn.metrics import precision_recall_curve

In [2]:
X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

In [3]:
X_test.head()

,HAEMATOCRIT,HAEMOGLOBINS,ERYTHROCYTE,LEUCOCYTE,THROMBOCYTE,MCH,MCHC,MCV,AGE,SEX
0,37.3,13.4,4.56,10.3,211,29.4,35.9,81.8,69,F
1,43.0,15.0,5.06,12.8,284,29.6,34.9,85.0,17,F
2,44.4,15.2,4.98,4.3,213,30.5,34.2,89.2,53,M
3,32.7,10.4,3.84,6.9,252,27.1,31.8,85.2,83,M
4,34.8,11.6,3.68,2.6,50,31.5,33.3,94.6,46,F


In [4]:
with open('xgb_pipeline.dill', 'rb') as f:
    pipeline = dill.load(f)

In [5]:
pipeline

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('HAEMATOCRIT',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='HAEMATOCRIT'))])),
                                                ('HAEMOGLOBINS',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='HAEMOGLOBINS'))])),
                                                ('ERYTHROCYTE',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='ERYTHROCYTE'))])),
                                                ('LEUCOCYTE',
                                                 Pipeline(steps=[('selector',
                                                                  Numbe...
             

In [6]:
preds = pipeline.predict_proba(X_test)[:, 1]

pred_df = pd.DataFrame({'test_predictions': preds})
pred_df.to_csv("test_predictions.csv", index=None)

In [7]:
preds[:10]

array([0.2448779 , 0.04887148, 0.17637011, 0.5012444 , 0.9333019 ,
       0.32591227, 0.8038295 , 0.21038562, 0.46099055, 0.388969  ],
      dtype=float32)

In [8]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
ix = np.argmax(fscore)

print(f'Best Threshold = {thresholds[ix]}, F-Score = {fscore[ix]:.3f}, Precision = {precision[ix]:.3f}, Recall = {recall[ix]:.3f}')

Best Threshold = 0.25811153650283813, F-Score = 0.689, Precision = 0.590, Recall = 0.827
